# Investigate Menstrual data set

### Data source: Menstrual Health & Productivity Dataset
Link: https://www.kaggle.com/datasets/puspitachowdhury2/menstrual-health-dataset/data

Import packages

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Load datasets and check for cleaniness

## User dataset

In [49]:
users_df = pd.read_csv('User_Profile.csv')
users_df.head()

,user_id,state,age,bmi,diet_quality,exercise_frequency,sleep_hours,caffeine_intake,water_intake_liters,alcohol_consumption,smoking_status,birth_control_use,pcos_diagnosed,stress_score_baseline
0,U00001,Rhode Island,34,26.0,Good,5–6 days/week,5.4,1.5,2.0,Occasionally,No,1,1,4.1
1,U00002,New Hampshire,32,26.6,Good,5–6 days/week,5.5,2.9,2.3,Weekly,No,0,0,5.7
2,U00003,Iowa,18,16.0,Poor,1–2 days/week,6.7,0.5,2.1,Never,No,1,0,3.1
3,U00004,Texas,30,24.7,Excellent,NaN,7.3,0.4,1.8,Occasionally,No,1,0,6.6
4,U00005,Delaware,32,29.2,Good,1–2 days/week,7.1,2.2,2.2,Occasionally,No,0,1,3.1


In [50]:
#identify number of rows and columns in the dataset
users_df.shape

(2000, 14)

This dataset has 2000 rows and 14 columns

In [51]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                2000 non-null   object 
 1   state                  2000 non-null   object 
 2   age                    2000 non-null   int64  
 3   bmi                    2000 non-null   float64
 4   diet_quality           2000 non-null   object 
 5   exercise_frequency     1615 non-null   object 
 6   sleep_hours            2000 non-null   float64
 7   caffeine_intake        2000 non-null   float64
 8   water_intake_liters    2000 non-null   float64
 9   alcohol_consumption    2000 non-null   object 
 10  smoking_status         2000 non-null   object 
 11  birth_control_use      2000 non-null   int64  
 12  pcos_diagnosed         2000 non-null   int64  
 13  stress_score_baseline  2000 non-null   float64
dtypes: float64(5), int64(3), object(6)
memory usage: 218.9+ 

Majority of the columns have no nulls, however exercise_frequency should be reviewed
I will want to change the boolean value columns to boolean data types

In [52]:
users_df.describe()

,age,bmi,sleep_hours,caffeine_intake,water_intake_liters,birth_control_use,pcos_diagnosed,stress_score_baseline
count,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000
mean,25.77500,22.984950,6.994200,1.816100,2.303700,0.34550,0.082000,5.588350
std,4.90259,4.229878,1.180696,1.062377,0.689764,0.47565,0.274433,1.471255
min,16.00000,16.000000,4.500000,0.000000,0.800000,0.00000,0.000000,1.000000
25%,22.00000,19.800000,6.200000,1.000000,1.800000,0.00000,0.000000,4.600000
50%,26.00000,22.800000,7.000000,1.800000,2.300000,0.00000,0.000000,5.600000
75%,29.00000,26.000000,7.800000,2.500000,2.800000,1.00000,0.000000,6.700000
max,43.00000,41.200000,10.000000,5.600000,4.500000,1.00000,1.000000,10.000000


The columns with 0 values are boolean values, so 0 values are expected

#### Identify and remove nulls

In [53]:
#count of null values by column
users_df.isnull().sum()

user_id                    0
state                      0
age                        0
bmi                        0
diet_quality               0
exercise_frequency       385
sleep_hours                0
caffeine_intake            0
water_intake_liters        0
alcohol_consumption        0
smoking_status             0
birth_control_use          0
pcos_diagnosed             0
stress_score_baseline      0
dtype: int64

In [54]:
#drop rows that have a null value for exercise_frequency
users_df.dropna(subset=['exercise_frequency'], inplace=True)
#confirm rows with null values were dropped
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1615 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                1615 non-null   object 
 1   state                  1615 non-null   object 
 2   age                    1615 non-null   int64  
 3   bmi                    1615 non-null   float64
 4   diet_quality           1615 non-null   object 
 5   exercise_frequency     1615 non-null   object 
 6   sleep_hours            1615 non-null   float64
 7   caffeine_intake        1615 non-null   float64
 8   water_intake_liters    1615 non-null   float64
 9   alcohol_consumption    1615 non-null   object 
 10  smoking_status         1615 non-null   object 
 11  birth_control_use      1615 non-null   int64  
 12  pcos_diagnosed         1615 non-null   int64  
 13  stress_score_baseline  1615 non-null   float64
dtypes: float64(5), int64(3), object(6)
memory usage: 189.3+ KB


### Review, update or drop categorical columns

#### Diet quality

In [55]:
#identify values in column
diet_quality = users_df[['diet_quality']].copy()
group_diet_quality = diet_quality.groupby('diet_quality').size()
group_diet_quality.head()

diet_quality
Average      565
Excellent    152
Good         745
Poor         153
dtype: int64

In [56]:
#ordinal data
#create order for values
diet_quality_order = {'Poor': 0, 'Average': 1, 'Good': 2, 'Excellent': 3}
#map to identified order
users_df['diet_quality_encoded'] = users_df['diet_quality'].map(diet_quality_order)
users_df.head()

,user_id,state,age,bmi,diet_quality,exercise_frequency,sleep_hours,caffeine_intake,water_intake_liters,alcohol_consumption,smoking_status,birth_control_use,pcos_diagnosed,stress_score_baseline,diet_quality_encoded
0,U00001,Rhode Island,34,26.0,Good,5–6 days/week,5.4,1.5,2.0,Occasionally,No,1,1,4.1,2
1,U00002,New Hampshire,32,26.6,Good,5–6 days/week,5.5,2.9,2.3,Weekly,No,0,0,5.7,2
2,U00003,Iowa,18,16.0,Poor,1–2 days/week,6.7,0.5,2.1,Never,No,1,0,3.1,0
4,U00005,Delaware,32,29.2,Good,1–2 days/week,7.1,2.2,2.2,Occasionally,No,0,1,3.1,2
5,U00006,Massachusetts,28,21.4,Average,3–4 days/week,7.6,1.5,1.9,Occasionally,No,1,0,4.8,1


#### Exercise frequency

In [57]:
#identify values in column
exercise_frequency = users_df[['exercise_frequency']].copy()
group_exercise_frequency = exercise_frequency.groupby('exercise_frequency').size()
group_exercise_frequency.head()

exercise_frequency
1–2 days/week    605
3–4 days/week    734
5–6 days/week    276
dtype: int64

In [58]:
#ordinal data
#create order for values
exercise_frequency_order = {'1–2 days/week': 0, '3–4 days/week': 1, '5–6 days/week': 2}
#map to identified order
users_df['exercise_frequency_encoded'] = users_df['exercise_frequency'].map(exercise_frequency_order)
users_df.head()

,user_id,state,age,bmi,diet_quality,exercise_frequency,sleep_hours,caffeine_intake,water_intake_liters,alcohol_consumption,smoking_status,birth_control_use,pcos_diagnosed,stress_score_baseline,diet_quality_encoded,exercise_frequency_encoded
0,U00001,Rhode Island,34,26.0,Good,5–6 days/week,5.4,1.5,2.0,Occasionally,No,1,1,4.1,2,2
1,U00002,New Hampshire,32,26.6,Good,5–6 days/week,5.5,2.9,2.3,Weekly,No,0,0,5.7,2,2
2,U00003,Iowa,18,16.0,Poor,1–2 days/week,6.7,0.5,2.1,Never,No,1,0,3.1,0,0
4,U00005,Delaware,32,29.2,Good,1–2 days/week,7.1,2.2,2.2,Occasionally,No,0,1,3.1,2,0
5,U00006,Massachusetts,28,21.4,Average,3–4 days/week,7.6,1.5,1.9,Occasionally,No,1,0,4.8,1,1


#### Alcohol consumption

In [59]:
#identify values in column
alcohol_consumption = users_df[['alcohol_consumption']].copy()
group_alcohol_consumption = alcohol_consumption.groupby('alcohol_consumption').size()
group_alcohol_consumption.head()

alcohol_consumption
Never           752
Occasionally    696
Weekly          167
dtype: int64

In [60]:
#ordinal data
#create order for values
alcohol_consumption_order = {'Never': 0, 'Occasionally': 1, 'Weekly': 2}
#map to identified order
users_df['alcohol_consumption_endcoded'] = users_df['alcohol_consumption'].map(alcohol_consumption_order)
users_df.head()

,user_id,state,age,bmi,diet_quality,exercise_frequency,sleep_hours,caffeine_intake,water_intake_liters,alcohol_consumption,smoking_status,birth_control_use,pcos_diagnosed,stress_score_baseline,diet_quality_encoded,exercise_frequency_encoded,alcohol_consumption_endcoded
0,U00001,Rhode Island,34,26.0,Good,5–6 days/week,5.4,1.5,2.0,Occasionally,No,1,1,4.1,2,2,1
1,U00002,New Hampshire,32,26.6,Good,5–6 days/week,5.5,2.9,2.3,Weekly,No,0,0,5.7,2,2,2
2,U00003,Iowa,18,16.0,Poor,1–2 days/week,6.7,0.5,2.1,Never,No,1,0,3.1,0,0,0
4,U00005,Delaware,32,29.2,Good,1–2 days/week,7.1,2.2,2.2,Occasionally,No,0,1,3.1,2,0,1
5,U00006,Massachusetts,28,21.4,Average,3–4 days/week,7.6,1.5,1.9,Occasionally,No,1,0,4.8,1,1,1


#### Smoking status

In [61]:
#identify values in column
smoking_status = users_df[['smoking_status']].copy()
group_smoking_status = smoking_status.groupby('smoking_status').size()
group_smoking_status.head()

smoking_status
No     1450
Yes     165
dtype: int64

In [62]:
#convert smoking_status to numerical
users_df['smoking_status_numeric'] = users_df['smoking_status'].replace({'Yes': 1, 'No': 0}).astype(int)
users_df.head()

/var/folders/l2/m9gp7xjj0z1cyk2v4gfx4h680000gn/T/ipykernel_81601/1152260525.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users_df['smoking_status_numeric'] = users_df['smoking_status'].replace({'Yes': 1, 'No': 0}).astype(int)


,user_id,state,age,bmi,diet_quality,exercise_frequency,sleep_hours,caffeine_intake,water_intake_liters,alcohol_consumption,smoking_status,birth_control_use,pcos_diagnosed,stress_score_baseline,diet_quality_encoded,exercise_frequency_encoded,alcohol_consumption_endcoded,smoking_status_numeric
0,U00001,Rhode Island,34,26.0,Good,5–6 days/week,5.4,1.5,2.0,Occasionally,No,1,1,4.1,2,2,1,0
1,U00002,New Hampshire,32,26.6,Good,5–6 days/week,5.5,2.9,2.3,Weekly,No,0,0,5.7,2,2,2,0
2,U00003,Iowa,18,16.0,Poor,1–2 days/week,6.7,0.5,2.1,Never,No,1,0,3.1,0,0,0,0
4,U00005,Delaware,32,29.2,Good,1–2 days/week,7.1,2.2,2.2,Occasionally,No,0,1,3.1,2,0,1,0
5,U00006,Massachusetts,28,21.4,Average,3–4 days/week,7.6,1.5,1.9,Occasionally,No,1,0,4.8,1,1,1,0


#### Drop encoded categorical

In [63]:
#drop categorical columns
users_df = users_df.drop(['diet_quality', 'exercise_frequency', 'alcohol_consumption', 'smoking_status'], axis=1)
#validate columns are dropped
users_df.head()

,user_id,state,age,bmi,sleep_hours,caffeine_intake,water_intake_liters,birth_control_use,pcos_diagnosed,stress_score_baseline,diet_quality_encoded,exercise_frequency_encoded,alcohol_consumption_endcoded,smoking_status_numeric
0,U00001,Rhode Island,34,26.0,5.4,1.5,2.0,1,1,4.1,2,2,1,0
1,U00002,New Hampshire,32,26.6,5.5,2.9,2.3,0,0,5.7,2,2,2,0
2,U00003,Iowa,18,16.0,6.7,0.5,2.1,1,0,3.1,0,0,0,0
4,U00005,Delaware,32,29.2,7.1,2.2,2.2,0,1,3.1,2,0,1,0
5,U00006,Massachusetts,28,21.4,7.6,1.5,1.9,1,0,4.8,1,1,1,0


#### State

The last categorical value in this dataset is state, I am dropping from this analysis

In [64]:
users_df = users_df.drop('state', axis=1) 
#validate columns are dropped
users_df.head()

,user_id,age,bmi,sleep_hours,caffeine_intake,water_intake_liters,birth_control_use,pcos_diagnosed,stress_score_baseline,diet_quality_encoded,exercise_frequency_encoded,alcohol_consumption_endcoded,smoking_status_numeric
0,U00001,34,26.0,5.4,1.5,2.0,1,1,4.1,2,2,1,0
1,U00002,32,26.6,5.5,2.9,2.3,0,0,5.7,2,2,2,0
2,U00003,18,16.0,6.7,0.5,2.1,1,0,3.1,0,0,0,0
4,U00005,32,29.2,7.1,2.2,2.2,0,1,3.1,2,0,1,0
5,U00006,28,21.4,7.6,1.5,1.9,1,0,4.8,1,1,1,0


#### Boolean

In [9]:
users_df['bc_use_bool'] = users_df['birth_control_use'].astype(bool)
users_df['pcos_bool'] = users_df['pcos_diagnosed'].astype(bool)

In [10]:
users_df.head()

,user_id,state,age,bmi,diet_quality,exercise_frequency,sleep_hours,caffeine_intake,water_intake_liters,alcohol_consumption,smoking_status,birth_control_use,pcos_diagnosed,stress_score_baseline,bc_use_bool,pcos_bool
0,U00001,Rhode Island,34,26.0,Good,5–6 days/week,5.4,1.5,2.0,Occasionally,No,1,1,4.1,True,True
1,U00002,New Hampshire,32,26.6,Good,5–6 days/week,5.5,2.9,2.3,Weekly,No,0,0,5.7,False,False
2,U00003,Iowa,18,16.0,Poor,1–2 days/week,6.7,0.5,2.1,Never,No,1,0,3.1,True,False
4,U00005,Delaware,32,29.2,Good,1–2 days/week,7.1,2.2,2.2,Occasionally,No,0,1,3.1,False,True
5,U00006,Massachusetts,28,21.4,Average,3–4 days/week,7.6,1.5,1.9,Occasionally,No,1,0,4.8,True,False


## Period Log Dataset

In [11]:
period_log_df = pd.read_csv('Period_Log.csv')
period_log_df.head()

,user_id,cycle_number,start_date,cycle_length_days,prev_cycle_length,cycle_phase,flow_level,pain_level,pms_symptoms,mood_score,...,sleep_hours_cycle,energy_level,concentration_score,work_hours_lost,estrogen_pgml,progesterone_ngml,ovulation_result,overall_health_score,log_consistency_score,prepared_before_period
0,U00001,1,2024-02-13,33,NaN,Luteal,Heavy,9,Yes,6,...,5.3,6,7,5.1,84.7,4.14,Positive,6.5,0.949,0
1,U00001,2,2024-03-17,33,33.0,Follicular,Light,2,No,8,...,4.6,7,8,2.6,126.1,2.79,Negative,9.4,0.949,1
2,U00001,3,2024-04-19,34,33.0,Follicular,Heavy,8,No,5,...,5.1,5,5,6.8,131.1,1.70,Negative,6.7,0.949,1
3,U00001,4,2024-05-23,31,34.0,Luteal,Light,3,No,8,...,6.7,8,8,2.5,82.0,8.50,Negative,9.0,0.949,0
4,U00001,5,2024-06-23,31,31.0,Luteal,Moderate,3,No,9,...,5.8,9,8,2.4,55.5,6.30,Negative,9.3,0.949,1


In [12]:
period_log_df.shape

(17976, 21)

In [13]:
period_log_df.describe()

,cycle_number,cycle_length_days,prev_cycle_length,pain_level,mood_score,stress_score_cycle,sleep_hours_cycle,energy_level,concentration_score,work_hours_lost,estrogen_pgml,progesterone_ngml,overall_health_score,log_consistency_score,prepared_before_period
count,17976.000000,17976.000000,15976.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000,17976.000000
mean,5.214842,27.816756,27.826865,4.854473,7.084947,5.749260,6.960253,6.832165,7.352025,3.394443,96.429840,5.336521,8.548665,0.935288,0.567701
std,2.931241,2.400329,2.399832,2.230292,1.506431,1.771953,1.315146,1.687533,1.435035,1.495181,27.928743,3.791151,1.106405,0.020129,0.495409
min,1.000000,22.000000,22.000000,1.000000,1.000000,1.000000,4.500000,1.000000,2.000000,0.000000,10.000000,0.100000,4.400000,0.860000,0.000000
25%,3.000000,26.000000,26.000000,3.000000,6.000000,4.500000,6.000000,6.000000,6.000000,2.300000,76.300000,2.230000,7.800000,0.921000,0.000000
50%,5.000000,28.000000,28.000000,5.000000,7.000000,5.800000,7.000000,7.000000,7.000000,3.400000,94.300000,3.830000,8.600000,0.936000,1.000000
75%,7.000000,29.000000,29.000000,6.000000,8.000000,7.000000,7.900000,8.000000,8.000000,4.400000,115.100000,8.370000,9.500000,0.950000,1.000000
max,12.000000,44.000000,44.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,8.800000,206.500000,20.250000,10.000000,0.985000,1.000000


In [14]:
period_log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17976 entries, 0 to 17975
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 17976 non-null  object 
 1   cycle_number            17976 non-null  int64  
 2   start_date              17976 non-null  object 
 3   cycle_length_days       17976 non-null  int64  
 4   prev_cycle_length       15976 non-null  float64
 5   cycle_phase             17976 non-null  object 
 6   flow_level              17976 non-null  object 
 7   pain_level              17976 non-null  int64  
 8   pms_symptoms            17976 non-null  object 
 9   mood_score              17976 non-null  int64  
 10  stress_score_cycle      17976 non-null  float64
 11  sleep_hours_cycle       17976 non-null  float64
 12  energy_level            17976 non-null  int64  
 13  concentration_score     17976 non-null  int64  
 14  work_hours_lost         17976 non-null

In [15]:
#convert start date column to date
period_log_df['start_date'] = pd.to_datetime(period_log_df['start_date'])
#confirm conversion
period_log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17976 entries, 0 to 17975
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   user_id                 17976 non-null  object        
 1   cycle_number            17976 non-null  int64         
 2   start_date              17976 non-null  datetime64[ns]
 3   cycle_length_days       17976 non-null  int64         
 4   prev_cycle_length       15976 non-null  float64       
 5   cycle_phase             17976 non-null  object        
 6   flow_level              17976 non-null  object        
 7   pain_level              17976 non-null  int64         
 8   pms_symptoms            17976 non-null  object        
 9   mood_score              17976 non-null  int64         
 10  stress_score_cycle      17976 non-null  float64       
 11  sleep_hours_cycle       17976 non-null  float64       
 12  energy_level            17976 non-null  int64 

In [16]:
#count of null values by column
period_log_df.isnull().sum()

user_id                      0
cycle_number                 0
start_date                   0
cycle_length_days            0
prev_cycle_length         2000
cycle_phase                  0
flow_level                   0
pain_level                   0
pms_symptoms                 0
mood_score                   0
stress_score_cycle           0
sleep_hours_cycle            0
energy_level                 0
concentration_score          0
work_hours_lost              0
estrogen_pgml                0
progesterone_ngml            0
ovulation_result             0
overall_health_score         0
log_consistency_score        0
prepared_before_period       0
dtype: int64

In [17]:
filtered_period_log_df = period_log_df[period_log_df['prev_cycle_length'].isnull()]
filtered_period_log_df.head()

,user_id,cycle_number,start_date,cycle_length_days,prev_cycle_length,cycle_phase,flow_level,pain_level,pms_symptoms,mood_score,...,sleep_hours_cycle,energy_level,concentration_score,work_hours_lost,estrogen_pgml,progesterone_ngml,ovulation_result,overall_health_score,log_consistency_score,prepared_before_period
0,U00001,1,2024-02-13,33,NaN,Luteal,Heavy,9,Yes,6,...,5.3,6,7,5.1,84.7,4.14,Positive,6.5,0.949,0
11,U00002,1,2024-03-13,25,NaN,Follicular,Light,2,No,9,...,5.6,8,9,2.6,63.9,3.44,Positive,9.6,0.923,0
23,U00003,1,2024-03-15,26,NaN,Menstrual,Moderate,4,No,8,...,7.3,7,9,2.5,27.4,1.47,Negative,8.9,0.953,0
34,U00004,1,2024-02-01,28,NaN,Follicular,Light,3,No,9,...,7.1,6,8,2.7,107.6,1.20,Negative,9.5,0.946,0
45,U00005,1,2024-03-23,36,NaN,Follicular,Moderate,5,No,6,...,7.6,10,7,2.0,104.7,2.58,Negative,8.9,0.929,0


In [18]:
#check if any of that the rows with na prev cycle length has the cycle number of 1
filtered_period_log_df = filtered_period_log_df[filtered_period_log_df['cycle_number'] != 1]
filtered_period_log_df.head()

,user_id,cycle_number,start_date,cycle_length_days,prev_cycle_length,cycle_phase,flow_level,pain_level,pms_symptoms,mood_score,...,sleep_hours_cycle,energy_level,concentration_score,work_hours_lost,estrogen_pgml,progesterone_ngml,ovulation_result,overall_health_score,log_consistency_score,prepared_before_period


In [19]:
filtered_period_log_df.describe()

,cycle_number,start_date,cycle_length_days,prev_cycle_length,pain_level,mood_score,stress_score_cycle,sleep_hours_cycle,energy_level,concentration_score,work_hours_lost,estrogen_pgml,progesterone_ngml,overall_health_score,log_consistency_score,prepared_before_period
count,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#merge datasets
merged_df = period_log_df.merge(users_df, on='user_id', how='inner')
merged_df.head()

,user_id,cycle_number,start_date,cycle_length_days,prev_cycle_length,cycle_phase,flow_level,pain_level,pms_symptoms,mood_score,...,sleep_hours,caffeine_intake,water_intake_liters,alcohol_consumption,smoking_status,birth_control_use,pcos_diagnosed,stress_score_baseline,bc_use_bool,pcos_bool
0,U00001,1,2024-02-13,33,NaN,Luteal,Heavy,9,Yes,6,...,5.4,1.5,2.0,Occasionally,No,1,1,4.1,True,True
1,U00001,2,2024-03-17,33,33.0,Follicular,Light,2,No,8,...,5.4,1.5,2.0,Occasionally,No,1,1,4.1,True,True
2,U00001,3,2024-04-19,34,33.0,Follicular,Heavy,8,No,5,...,5.4,1.5,2.0,Occasionally,No,1,1,4.1,True,True
3,U00001,4,2024-05-23,31,34.0,Luteal,Light,3,No,8,...,5.4,1.5,2.0,Occasionally,No,1,1,4.1,True,True
4,U00001,5,2024-06-23,31,31.0,Luteal,Moderate,3,No,9,...,5.4,1.5,2.0,Occasionally,No,1,1,4.1,True,True


In [28]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14490 entries, 0 to 14489
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   user_id                 14490 non-null  object        
 1   cycle_number            14490 non-null  int64         
 2   start_date              14490 non-null  datetime64[ns]
 3   cycle_length_days       14490 non-null  int64         
 4   prev_cycle_length       12875 non-null  float64       
 5   cycle_phase             14490 non-null  object        
 6   flow_level              14490 non-null  object        
 7   pain_level              14490 non-null  int64         
 8   pms_symptoms            14490 non-null  object        
 9   mood_score              14490 non-null  int64         
 10  stress_score_cycle      14490 non-null  float64       
 11  sleep_hours_cycle       14490 non-null  float64       
 12  energy_level            14490 non-null  int64 

In [29]:
#group by user to create a dataset grouped by averages for each users
grouped_merged_df = merged_df.groupby('user_id').mean(numeric_only=True)
grouped_merged_df

,cycle_number,cycle_length_days,prev_cycle_length,pain_level,mood_score,stress_score_cycle,sleep_hours_cycle,energy_level,concentration_score,work_hours_lost,...,age,bmi,sleep_hours,caffeine_intake,water_intake_liters,birth_control_use,pcos_diagnosed,stress_score_baseline,bc_use_bool,pcos_bool
user_id,,,,,,,,,,,,,,,,,,,,,
U00001,6.0,32.909091,32.900000,5.272727,7.090909,4.518182,5.463636,7.181818,7.636364,4.018182,...,34.0,26.0,5.4,1.5,2.0,1.0,1.0,4.1,1.0,1.0
U00002,6.5,27.583333,27.636364,4.833333,7.333333,5.641667,5.475000,7.166667,7.083333,4.125000,...,32.0,26.6,5.5,2.9,2.3,0.0,0.0,5.7,0.0,0.0
U00003,6.0,27.000000,27.100000,3.181818,8.545455,3.781818,6.981818,8.181818,8.636364,2.309091,...,18.0,16.0,6.7,0.5,2.1,1.0,0.0,3.1,1.0,0.0
U00005,3.5,35.000000,35.200000,3.666667,7.500000,2.683333,7.500000,9.000000,8.333333,1.900000,...,32.0,29.2,7.1,2.2,2.2,0.0,1.0,3.1,0.0,1.0
U00006,5.5,26.800000,26.666667,3.900000,7.200000,5.460000,7.640000,7.400000,7.700000,2.760000,...,28.0,21.4,7.6,1.5,1.9,1.0,0.0,4.8,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U01995,4.0,27.285714,27.166667,4.428571,7.000000,6.700000,4.942857,6.571429,7.714286,3.828571,...,29.0,18.4,4.6,1.4,1.9,1.0,0.0,7.1,1.0,0.0
U01997,3.5,27.500000,27.600000,2.000000,8.500000,1.983333,7.216667,9.166667,9.333333,1.116667,...,30.0,32.8,6.9,1.5,1.3,0.0,0.0,2.4,0.0,0.0
U01998,5.5,29.300000,29.111111,4.700000,7.400000,6.080000,9.070000,6.800000,7.800000,2.270000,...,27.0,26.1,9.2,2.1,2.8,1.0,0.0,6.4,1.0,0.0


In [31]:
grouped_merged_df = merged_df.groupby('user_id').mean(numeric_only=True)
grouped_merged_df

,cycle_number,cycle_length_days,prev_cycle_length,pain_level,mood_score,stress_score_cycle,sleep_hours_cycle,energy_level,concentration_score,work_hours_lost,...,age,bmi,sleep_hours,caffeine_intake,water_intake_liters,birth_control_use,pcos_diagnosed,stress_score_baseline,bc_use_bool,pcos_bool
user_id,,,,,,,,,,,,,,,,,,,,,
U00001,6.0,32.909091,32.900000,5.272727,7.090909,4.518182,5.463636,7.181818,7.636364,4.018182,...,34.0,26.0,5.4,1.5,2.0,1.0,1.0,4.1,1.0,1.0
U00002,6.5,27.583333,27.636364,4.833333,7.333333,5.641667,5.475000,7.166667,7.083333,4.125000,...,32.0,26.6,5.5,2.9,2.3,0.0,0.0,5.7,0.0,0.0
U00003,6.0,27.000000,27.100000,3.181818,8.545455,3.781818,6.981818,8.181818,8.636364,2.309091,...,18.0,16.0,6.7,0.5,2.1,1.0,0.0,3.1,1.0,0.0
U00005,3.5,35.000000,35.200000,3.666667,7.500000,2.683333,7.500000,9.000000,8.333333,1.900000,...,32.0,29.2,7.1,2.2,2.2,0.0,1.0,3.1,0.0,1.0
U00006,5.5,26.800000,26.666667,3.900000,7.200000,5.460000,7.640000,7.400000,7.700000,2.760000,...,28.0,21.4,7.6,1.5,1.9,1.0,0.0,4.8,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U01995,4.0,27.285714,27.166667,4.428571,7.000000,6.700000,4.942857,6.571429,7.714286,3.828571,...,29.0,18.4,4.6,1.4,1.9,1.0,0.0,7.1,1.0,0.0
U01997,3.5,27.500000,27.600000,2.000000,8.500000,1.983333,7.216667,9.166667,9.333333,1.116667,...,30.0,32.8,6.9,1.5,1.3,0.0,0.0,2.4,0.0,0.0
U01998,5.5,29.300000,29.111111,4.700000,7.400000,6.080000,9.070000,6.800000,7.800000,2.270000,...,27.0,26.1,9.2,2.1,2.8,1.0,0.0,6.4,1.0,0.0
